<a href="https://colab.research.google.com/github/conradtav/Portfolio/blob/master/LAPD_911.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [49]:
LAPD = pd.read_csv("/content/LAPD_Calls_for_Service_2020.csv").sample(n=100000, random_state=12, replace=True)
LAPD = LAPD[LAPD['Area_Occ']!='Outside']

LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('N Hollywood', 'N Hollywood NORTHEAST', regex=True)
LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('Devonshire', 'Northridge', regex=True)
LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('77th Street', 'South Central', regex=True)
LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('Pacific', 'Mar Vista', regex=True)
LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('Newton', 'Alameda', regex=True)
LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('Northeast', 'Atwater', regex=True)
LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('West Valley', 'Encino', regex=True)
LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('Hollenbeck', 'Boyle Heights', regex=True)
LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('West LA', 'Westwood', regex=True)
LAPD['Area_Occ'] = LAPD['Area_Occ'].replace('Central', 'Downtown Los Angeles', regex=True)


LAPD.head()

,Incident_Number,Area_Occ,Rpt_Dist,Dispatch_Date,Dispatch_Time,Call_Type_Code,Call_Type_Text
472907,PD20021100001866,Rampart,216.0,02/11/2020 12:00:00 AM,09:48:14,242A,AMB
759451,PD20052600005147,N Hollywood NORTHEAST,1549.0,05/26/2020 12:00:00 AM,19:49:33,9212,TRESPASS SUSP
412925,PD20050800000797,Southwest,341.0,05/08/2020 12:00:00 AM,06:39:15,9032,HAZARD
564465,PD20032000001222,Atwater,1142.0,03/20/2020 12:00:00 AM,09:15:10,906B1,CODE 30 RINGER
915395,PD20062800004479,Northridge,1735.0,06/28/2020 12:00:00 AM,20:31:04,507F,FIREWORKS


In [50]:
census = pd.read_csv("/content/Census_Data_by_Neighborhood_Council.csv")
#census['NC_Name'] = 
census['NC_Name'] = census['NC_Name'].str[:-3]

In [33]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [51]:
import pandas as pd 
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
           names_array.append(wrong_option)
           ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

In [52]:
str2Match = LAPD['Area_Occ'].fillna('######').tolist()

strOptions =census['NC_Name'].fillna('######').tolist()

In [53]:

name_match, ratio_match=checker(str2Match,strOptions)

s1=pd.Series(str2Match)

s2=pd.Series(name_match)

s3=pd.Series(ratio_match)


In [54]:
LAPD['Area_Occ'].unique()

array(['Rampart', 'N Hollywood NORTHEAST', 'Southwest', 'Atwater',
       'Northridge', 'Foothill', 'Southeast', 'Mission', 'Olympic',
       'Encino', 'Boyle Heights', 'Alameda', 'Wilshire',
       'Downtown Los Angeles', 'Mar Vista', 'Topanga', 'Hollywood',
       'South Downtown Los Angeles', 'Harbor', 'Westwood', 'Van Nuys'],
      dtype=object)

In [55]:
matches = pd.DataFrame([s1, s2, s3])
matches.T.drop_duplicates()

,0,1,2
0,Rampart,RAMPART VILLAGE,100
1,N Hollywood NORTHEAST,NORTH HOLLYWOOD NORTHEAST,95
2,Southwest,EMPOWERMENT CONGRESS SOUTHWEST AREA,100
3,Atwater,ATWATER VILLAGE,100
4,Northridge,NORTHRIDGE E,100
5,Foothill,FOOTHILL TRAILS DISTRICT,100
6,Southeast,EMPOWERMENT CONGRESS SOUTHEAST AREA,100
8,Mission,MISSION HILLS,100
9,Olympic,OLYMPIC PARK,100
10,Encino,ENCINO,100


In [56]:
#pd.concat(LAPD, s1, s2, s3, ignore_index = True)
LAPD.reset_index(drop=True, inplace=True)
s1.reset_index(drop=True, inplace=True)
LAPD = pd.concat([LAPD, s1, s2, s3], axis=1, ignore_index=True)


,0,1,2,3,4,5,6,7,8,9
0,PD20021100001866,Rampart,216.0,02/11/2020 12:00:00 AM,09:48:14,242A,AMB,Rampart,RAMPART VILLAGE,100
1,PD20052600005147,N Hollywood NORTHEAST,1549.0,05/26/2020 12:00:00 AM,19:49:33,9212,TRESPASS SUSP,N Hollywood NORTHEAST,NORTH HOLLYWOOD NORTHEAST,95
2,PD20050800000797,Southwest,341.0,05/08/2020 12:00:00 AM,06:39:15,9032,HAZARD,Southwest,EMPOWERMENT CONGRESS SOUTHWEST AREA,100
3,PD20032000001222,Atwater,1142.0,03/20/2020 12:00:00 AM,09:15:10,906B1,CODE 30 RINGER,Atwater,ATWATER VILLAGE,100
4,PD20062800004479,Northridge,1735.0,06/28/2020 12:00:00 AM,20:31:04,507F,FIREWORKS,Northridge,NORTHRIDGE E,100


In [68]:
LAPD.columns = ['Id', 'Incident_Number	Area_Occ',	'Rpt_Dist',	'Dispatch_Date',	'Dispatch_Time',	'Call_Type_Code',	'Call_Type_Text', 'original', 'Match', "Match_Score"]
LAPD.merge(LAPD, census, how='left', left_on='Match', right_on='NC_Name')


TypeError: ignored